# Initialization

In [74]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

col_pal = sns.color_palette()

In [65]:
df = pd.read_csv('https://raw.githubusercontent.com/hemanthhariharan/CS_229_Project/main/T1.csv')

df['Date/Time'] = pd.to_datetime(df['Date/Time'], format = "%d %m %Y %H:%M", errors = "coerce")

df = df.set_index('Date/Time')

# Creating time-series features

df['hour'] = df.index.hour
df['day_of_week'] = df.index.day_of_week

df_normalized = (df - df.mean())/ df.std()

mean_target = df['Theoretical_Power_Curve (KWh)'].mean()

std_target = df['Theoretical_Power_Curve (KWh)'].std()

In [66]:
train = df_normalized.loc[df.index < '2018-12-01'] # train on 11 months
test = df_normalized.loc[df.index >= '2018-12-01'] # test on 1 month

features = ['LV ActivePower (kW)', 'Wind Speed (m/s)', 'Wind Direction (°)', 'hour', 'day_of_week']
target = ['Theoretical_Power_Curve (KWh)']

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

In [67]:
# batch_size = 1
# epoch = 15
# neurons = 10
# predict_values = 1000
# lag = 24

# X_train_lstm = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])

# model = Sequential()
# model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')

# LSTM

In [93]:
neurons = 200
batch_size = 100
epochs = 50
dropout = .05

X_train_lstm = X_train.to_numpy()

X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], 1, X_train_lstm.shape[1])

# lstm_model = Sequential()
# lstm_model.add(LSTM(units=neurons, dropout=dropout, activation='relu', batch_input_shape=(batch_size, X_train_lstm.shape[1], X_train_lstm.shape[2])))
# lstm_model.add(Dense(1))  # Output layer for regression tasks
# lstm_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# lstm_model.summary()

lstm_model = LSTM(units=neurons, dropout=dropout, activation='relu', batch_input_shape=(batch_size, X_train_lstm.shape[1], X_train_lstm.shape[2]))
# lstm_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# XGBoost

In [69]:
xgbreg = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=100, learning_rate=0.01, max_depth=5, reg_lambda=1)

# xgbmodel = xgb.XGBRegressor()

# Ensemble

In [ ]:
class LSTMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = Sequential()
        neurons = 200
        batch_size = 100
        epochs = 50
        dropout = .05
        # Add LSTM layers and other necessary layers

    def fit(self, X, y):
        self.add(LSTM(units=neurons, dropout=dropout, activation='relu', batch_input_shape=(batch_size, X_train_lstm.shape[1], X_train_lstm.shape[2])))
        self.add(Dense(1))  # Output layer for regression tasks
        self.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
        # Implement your fit logic here

    def predict(self, X):
        # Implement your prediction logic here
        return self.model.predict(X)

In [96]:
# estimator_list =  [('lr', LinearRegression()), ('lstm', lstm_model), ('xgb', xgbreg)]

estimator_list =  [('lr', LinearRegression()), ('logreg', LogisticRegression())]

ensemble = StackingRegressor(estimators = estimator_list, final_estimator=LogisticRegression())

# ensemble = StackingClassifier(estimators = 
#                               [('lstm', lstm_model)]
#                              )

# ensemble = StackingRegressor(estimators = 
#                               [('lr', LinearRegression()),
#                                ('xgb', xgbreg)],
#                               final_estimator=LinearRegression())

In [97]:
ensemble.fit(X_train, y_train)

/opt/homebrew/lib/python3.10/site-packages/sklearn/ensemble/_stacking.py:955: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: The estimator LogisticRegression should be a regressor.